In [11]:
import sqlite3
import pandas as pd
import altair as alt
import os

In [12]:
# Load cleaned CSVs
primary = pd.read_csv("data/cleaned_data/primary.csv")
presidential = pd.read_csv("data/cleaned_data/presidentialResult.csv")

# Connect to SQLite DB (will create one if it doesn't exist)
conn = sqlite3.connect("research.db")


In [13]:
# Write data to tables
primary.to_sql("readmissions", conn, if_exists="replace", index=False)
presidential.to_sql("elections", conn, if_exists="replace", index=False)

# ✅ Ensure output folder exists
os.makedirs("output", exist_ok=True)

# Connect to the SQLite DB
conn = sqlite3.connect("research.db")

In [14]:
### RQ1: Change in voting pattern vs. change in readmission rates --- ###
query_rq1 = """
SELECT
    e2016.County,
    e2016.Win AS Win_2016,
    e2020.Win AS Win_2020,
    r2016."30_day_Readmission_Rate" AS Readmit_2016,
    r2020."30_day_Readmission_Rate" AS Readmit_2020,
    ROUND(r2020."30_day_Readmission_Rate" - r2016."30_day_Readmission_Rate", 2) AS Rate_Change
FROM elections e2016
JOIN elections e2020 ON e2016.County = e2020.County AND e2016.Year = 2016 AND e2020.Year = 2020
JOIN readmissions r2016 ON e2016.County = r2016.County AND r2016.Year = 2016
JOIN readmissions r2020 ON e2020.County = r2020.County AND r2020.Year = 2020;
"""
rq1_df = pd.read_sql_query(query_rq1, conn)
rq1_df.to_csv("output/rq1_voting_readmit_change.csv", index=False)

In [17]:
### RQ2: Average rate change for consistent vs. switched counties --- ###
query_rq2 = """
SELECT
    CASE
        WHEN e2016.Win = e2020.Win THEN 'Consistent'
        ELSE 'Switched'
    END AS Voting_Change,
    ROUND(AVG(r2020."30_day_Readmission_Rate" - r2016."30_day_Readmission_Rate"), 2) AS Avg_Readmit_Change
FROM elections e2016
JOIN elections e2020 ON e2016.County = e2020.County AND e2016.Year = 2016 AND e2020.Year = 2020
JOIN readmissions r2016 ON e2016.County = r2016.County AND r2016.Year = 2016
JOIN readmissions r2020 ON e2020.County = r2020.County AND r2020.Year = 2020
GROUP BY Voting_Change;
"""
rq2_df = pd.read_sql_query(query_rq2, conn)
rq2_df.to_csv("output/rq2_consistency_vs_rate_change.csv", index=False)

# Simple bar chart
alt.Chart(rq2_df).mark_bar().encode(
    x='Voting_Change:N',
    y='Avg_Readmit_Change:Q',
    color='Voting_Change:N'
).properties(
    title='Average Readmission Rate Change: Consistent vs Switched Counties'
)


alt.Chart(...)

In [18]:
### --- RQ3: Average readmission by political leaning and year --- ###
query_rq3 = """
SELECT
    e.Year,
    e.Win,
    ROUND(AVG(r."30_day_Readmission_Rate"), 2) AS Avg_Readmission_Rate
FROM elections e
JOIN readmissions r ON e.County = r.County AND e.Year = r.Year
GROUP BY e.Year, e.Win
ORDER BY e.Year, e.Win;
"""
rq3_df = pd.read_sql_query(query_rq3, conn)
rq3_df.to_csv("output/rq3_avg_readmit_by_party_year.csv", index=False)

# Grouped bar chart
alt.Chart(rq3_df).mark_bar().encode(
    x='Win:N',
    y='Avg_Readmission_Rate:Q',
    color='Win:N',
    column='Year:N'
).properties(
    title='Average Readmission Rate by Party (2016 vs 2020)'
)


alt.Chart(...)

In [19]:
# Close DB connection
conn.close()